In [338]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

In [339]:
# Constants

gammac = 1.4
gammah = 1.33

cpa = 1.005
cp = 1.148
R = 287

In [340]:
# Compressor Values

Ca4 = 226.4117 # m/s
Ca5 = Ca4
N = 208.05 # Rev/s
deltaTocomp = 439.58 # K

In [341]:
To4 = 2275 # K
polyt = 0.92
#flowcoeff Greater than 0.78
#loadcoeff Cannot Exceed 3.3
mfahot = 40 # kg/s
mfthot = mfahot + (mfahot*.02)

In [342]:
# Turbine PARAMETERS

#U4t = 290
htratioturbine = 0.5

In [343]:
deltaToturb = (deltaTocomp*cpa*0.99)/cp

T4 = To4-((Ca4**2)/(2*cp*1000))

po4 = 15 # bar
p4 = po4*(T4/To4)**(gammah/(gammah-1))

density4 = (p4*100000)/(R*T4)

print(density4)


2.22971091617132


In [344]:
A4 = mfthot/(density4*Ca4)

M4 = Ca4/(np.sqrt(1.333*R*To4))

print("Turbine Inlet Area = %3.4f m^2" % (A4))

print("Turbine Inlet Mach Number = %3.2f" % (M4))

Turbine Inlet Area = 0.0808 m^2
Turbine Inlet Mach Number = 0.24


In [345]:
r4t = np.sqrt(mfthot/(np.pi*density4*Ca4*(1-(htratioturbine**2))))

r4h = htratioturbine*r4t

rm = (r4t+r4h)/2

print("r4t = %3.3f m" % (r4t))
print("rm = %3.3f m" % (rm))
print("r4h = %3.3f m" % (r4h))

r4t = 0.185 m
rm = 0.139 m
r4h = 0.093 m


In [346]:
To5 = To4 - deltaToturb

T5 = To5 - (Ca5**2)/(2*cp*1000)

po5 = po4*((To5/To4)**((gammah*polyt)/(gammah-1)))

p5 = po5*(T5/To5)**(gammah/(gammah-1))

density5 = (p5*100000)/(R*T5)

A5 = mfthot/(density5*Ca5)

print("Turbine Exhaust Area = %3.4f m^2" % (A5))

Turbine Exhaust Area = 0.1336 m^2


In [347]:
bladeheight5 = A5/(2*np.pi*rm)

r5t = rm+(bladeheight5/2)

r5h = rm-(bladeheight5/2)

print("r5t = %3.3f m" % (r5t))
print("rm = %3.3f m" % (rm))
print("r5h = %3.3f m" % (r5h))

r5t = 0.215 m
rm = 0.139 m
r5h = 0.062 m


In [348]:
U5h = r5h*(2*np.pi*N)

print(U5h)

flow5 = Ca5/U5h

print(flow5)

bladeloading = 3.3

print(bladeloading)

deltaTos = (bladeloading*(U5h**2))/(2*cp*1000)

print(deltaTos)

stageest = deltaToturb/deltaTos

print("Estimation of %3.1f Stages" % (stageest))

81.55333256157691
2.776240932018905
3.3
9.55928657285158
Estimation of 39.9 Stages


In [349]:
U4t = 2*np.pi*r4t*N
U4m = 2*np.pi*rm*N
U4h = 2*np.pi*r4h*N

In [350]:
stageactual = 5
# Values per stage: alpha2, alpha3, beta2, beta3, r, deltaT, T3, U, Cw2, Cw3, V2, V3, pr, po3, bladeloading, flowcoeff
stagevals = np.zeros((stageactual+1,3,17))
stagevals[0,:,:] = np.array([[0,0,0,0,0,0,To4,U4h,Ca4,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4m,Ca4,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4t,Ca4,0,0,0,0,0,po4,0,0]])

$$
\psi = \frac{2 c_p \Delta T_{0s}}{U^2} \\
\phi = \frac{C_a}{U} \\
tan \beta_2 = \frac{1}{2 \phi} (\frac{\psi}{2} - 2\Lambda) \\
tan \beta_3 = \frac{1}{2 \phi} (\frac{\psi}{2} + 2\Lambda) \\
$$

In [351]:
def returnstage(deltaTo,U,Ca,DOR):
    bladeloading = (2*cp*1000*deltaTo)/(U**2)
    flow = Ca/U
    beta2 = np.degrees(np.arctan((1/(2*flow))*((bladeloading/2)-(2*DOR))))
    beta3 = np.degrees(np.arctan((1/(2*flow))*((bladeloading/2)+(2*DOR))))
    alpha2 = np.degrees(np.arctan(np.tan(np.radians(beta2))+(1/flow)))
    alpha3 = np.degrees(np.arctan((np.tan(np.radians(beta3))-(1/flow))))

    Cw2 = Ca*np.tan(np.radians(alpha2))
    Cw3 = Ca*np.tan(np.radians(alpha3))

    V2 = Ca/(np.cos(np.radians(beta2)))
    V3 = Ca/(np.cos(np.radians(beta3)))

    return beta2,beta3,alpha2,alpha3,Cw2,Cw3,V2,V3,bladeloading,flow

In [352]:
i = 1

deltaTosact = deltaToturb/stageactual

print(deltaTosact)

Cw1t = 0 # m/s
Cw1m = 0 # m/s
Cw1h = 0 # m/s

# This runs through the params for each stage and saves them to an array
while i < stageactual+1:
    if i == 0:
        DOR = 0.5
    if i == 1:
        DOR = 0.55
    if i == 2:
        DOR = 0.6
    if i == 3:
        DOR = 0.7
    else:
        DOR = 0.8

    rtstage = ((r5t-r4t)/(stageactual-1))*i+r4t 
    rhstage = ((r5h-r4h)/(stageactual-1))*i+r4h

    Utstage = 2*np.pi*rtstage*N
    Umstage = 2*np.pi*rm*N
    Uhstage = 2*np.pi*rhstage*N

    stageTo3 = stagevals[i-1,0,6] + deltaTosact

    # I PROBABLY NEED CHANGED
    pr = (1+((polyt*deltaTosact)/(stagevals[i-1,0,6])))**(gammac/(gammac-1))

    # DO NOT TOUCH THE COMMENTED FUNCTION UNLESS THE FIRST STAGE VALUES ARE FUCKED UP

    """if i == 1:
        deltawhirlt = ((1.005*(10**3)*(stagedeltaTo))/(lambcomp*Utstage))
        Cw2t = deltawhirlt - Cw1t
        stagebeta1tip = np.degrees(np.arctan(Utstage/Castage))
        stagebeta2tip = np.degrees(np.arctan((Utstage-Cw2t)/Castage))
        stagealpha1tip = 0
        stagealpha2tip = np.degrees(np.arctan((Cw2t)/Castage))

        V1t = Castage/(np.cos(np.radians(stagebeta1tip)))
        dehallert = np.cos(np.radians(stagebeta1tip))/np.cos(np.radians(stagebeta2tip))
        V2t = V1t*dehallert

        deltawhirlm = ((1.005*(10**3)*(stagedeltaTo))/(lambcomp*Umcomp))
        Cw2m = deltawhirlm - Cw1m
        stagebeta1mean = np.degrees(np.arctan(Umcomp/Castage))
        stagebeta2mean = np.degrees(np.arctan((Umcomp-Cw2m)/Castage))
        stagealpha1mean = 0
        stagealpha2mean = np.degrees(np.arctan((Cw2m)/Castage))

        V1m = Castage/(np.cos(np.radians(stagebeta1mean)))
        dehallerm = np.cos(np.radians(stagebeta1mean))/np.cos(np.radians(stagebeta2mean))
        V2m = V1m*dehallerm

        deltawhirlh = ((1.005*(10**3)*(stagedeltaTo))/(lambcomp*Uhcomp))
        Cw2h = deltawhirlh - Cw1h
        stagebeta1hub = np.degrees(np.arctan(Uhcomp/Castage))
        stagebeta2hub = np.degrees(np.arctan((Uhcomp-Cw2h)/Castage))
        stagealpha1hub = 0
        stagealpha2hub = np.degrees(np.arctan((Cw2h)/Castage))

        V1h = Castage/(np.cos(np.radians(stagebeta1hub)))
        dehallerh = np.cos(np.radians(stagebeta1hub))/np.cos(np.radians(stagebeta2hub))
        V2h = V1h*dehallerh
    else: """
    stagebeta2tip,stagebeta3tip,stagealpha2tip,stagealpha3tip,Cw2t,Cw3t,V2t,V3t,bladelt,flowt = returnstage(deltaTosact,Utstage,Ca4,DOR)

    stagebeta2mean,stagebeta3mean,stagealpha2mean,stagealpha3mean,Cw2m,Cw3m,V2m,V3m,bladelm,flowm = returnstage(deltaTosact,Umstage,Ca4,DOR)

    stagebeta2hub,stagebeta3hub,stagealpha2hub,stagealpha3hub,Cw2h,Cw3h,V2h,V3h,bladelh,flowh = returnstage(deltaTosact,Uhstage,Ca4,DOR)

    #pr = (1+((polyc*(((Umstage*Castage)/(cpa*1000))*(np.tan(np.radians(stagebeta1mean))-np.tan(np.radians(stagebeta2mean)))))/(stagevals[i-1,0,6])))**(gammac/(gammac-1))
    stagepo3 = stagevals[i-1,0,15]*pr

    stagevals[i,:,:] = np.array([[stagealpha2hub,stagealpha3hub,stagebeta2hub,stagebeta3hub,rhstage,deltaTosact,stageTo3,Uhstage,Ca4,Cw2h,Cw3h,V2h,V3h,pr,stagepo3,bladelh,flowh],[stagealpha2mean,stagealpha3mean,stagebeta2mean,stagebeta3mean,rm,deltaTosact,stageTo3,Umstage,Ca4,Cw2m,Cw3m,V2m,V3m,pr,stagepo3,bladelm,flowm],[stagealpha2tip,stagealpha3tip,stagebeta2tip,stagebeta3tip,rtstage,deltaTosact,stageTo3,Utstage,Ca4,Cw2t,Cw3t,V2t,V3t,pr,stagepo3,bladelt,flowt]])

    i += 1

76.19514303135888


In [353]:
def valtable(location,vals):
    if location == 0:
        print("Hub Values:")
    elif location == 1:
        print("Mean Values:")
    elif location == 2:
        print("Tip Values:")
    
    num = 0
    stages = ["Inlet"]

    while num < stageactual:
        stages.append("Stage "+str(num+1))
        num += 1

    pd.set_option("display.precision", 4)
    tabletry = pd.DataFrame({"Alpha 1": vals[:,location,0],"Alpha 2": vals[:,location,1],"Beta 1": vals[:,location,2],"Beta 2": vals[:,location,3],"Radius": vals[:,location,4],"Delta To": vals[:,location,5],"To2": vals[:,location,6],"U": vals[:,location,7],"Ca": vals[:,location,8],"Cw1": vals[:,location,9],"Cw2": vals[:,location,10],"V1": vals[:,location,11],"V2": vals[:,location,12],"PR": vals[:,location,13],"po2": vals[:,location,14],"Blade Loading": vals[:,location,15],"Flow": vals[:,location,16]},index=stages)

    return tabletry

In [354]:
valtable(0,stagevals)

Hub Values:


,Alpha 1,Alpha 2,Beta 1,Beta 2,Radius,Delta To,To2,U,Ca,Cw1,Cw2,V1,V2,PR,po2,Blade Loading,Flow
Inlet,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2275.0000,121.0510,226.4117,0.0000,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000
Stage 1,61.4208,58.6161,53.3629,64.8542,0.0850,76.1951,2351.1951,111.1766,226.4117,415.6276,371.1570,379.4110,532.8301,1.1121,0.0000,14.1538,2.0365
Stage 2,63.3931,61.1786,57.1535,66.1767,0.0775,76.1951,2427.3903,101.3022,226.4117,451.9986,411.4777,417.4329,560.5403,1.1083,15.6866,17.0475,2.2350
Stage 3,65.8850,63.3392,61.3477,67.3418,0.0699,76.1951,2503.5854,91.4278,226.4117,505.7953,450.9386,472.1892,587.7274,1.1048,18.8337,20.9287,2.4764
Stage 4,67.7200,66.4704,64.3283,69.3740,0.0624,76.1951,2579.7806,81.5533,226.4117,552.5979,519.9766,522.6330,642.7289,1.1015,23.0525,26.3036,2.7762
Stage 5,70.0720,69.1935,67.7282,71.2636,0.0548,76.1951,2655.9757,71.6789,226.4117,624.5014,595.8299,597.3902,704.8619,1.0984,28.8914,34.0499,3.1587


In [355]:
valtable(1,stagevals)

Mean Values:


,Alpha 1,Alpha 2,Beta 1,Beta 2,Radius,Delta To,To2,U,Ca,Cw1,Cw2,V1,V2,PR,po2,Blade Loading,Flow
Inlet,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2275.0000,181.5765,226.4117,0.0000,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000
Stage 1,50.7570,42.0964,22.8929,59.6143,0.1389,76.1951,2351.1951,181.5765,226.4117,277.1835,204.5529,245.7701,447.6139,1.1121,0.0000,5.3062,1.2469
Stage 2,50.7570,42.0964,22.8929,59.6143,0.1389,76.1951,2427.3903,181.5765,226.4117,277.1835,204.5529,245.7701,447.6139,1.1083,15.6866,5.3062,1.2469
Stage 3,52.5258,39.4632,26.6781,58.3962,0.1389,76.1951,2503.5854,181.5765,226.4117,295.3412,186.3953,253.3864,432.0480,1.1048,18.8337,5.3062,1.2469
Stage 4,50.7570,42.0964,22.8929,59.6143,0.1389,76.1951,2579.7806,181.5765,226.4117,277.1835,204.5529,245.7701,447.6139,1.1015,23.0525,5.3062,1.2469
Stage 5,50.7570,42.0964,22.8929,59.6143,0.1389,76.1951,2655.9757,181.5765,226.4117,277.1835,204.5529,245.7701,447.6139,1.0984,28.8914,5.3062,1.2469


In [356]:
valtable(2,stagevals)

Tip Values:


,Alpha 1,Alpha 2,Beta 1,Beta 2,Radius,Delta To,To2,U,Ca,Cw1,Cw2,V1,V2,PR,po2,Blade Loading,Flow
Inlet,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2275.0000,242.1020,226.4117,0.0000,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000
Stage 1,44.6890,28.5479,-7.0522,58.8882,0.1928,76.1951,2351.1951,251.9764,226.4117,223.9671,123.1766,228.1376,438.1803,1.1121,0.0000,2.7554,0.8985
Stage 2,44.0985,26.8579,-10.6202,58.9795,0.2003,76.1951,2427.3903,261.8508,226.4117,219.3966,114.6563,230.3576,439.3403,1.1083,15.6866,2.5515,0.8647
Stage 3,46.9620,19.3340,-7.3615,57.1883,0.2079,76.1951,2503.5854,271.7253,226.4117,242.4743,79.4392,228.2934,417.8262,1.1048,18.8337,2.3694,0.8332
Stage 4,43.0676,23.6161,-17.1725,59.2519,0.2154,76.1951,2579.7806,281.5997,226.4117,211.6327,98.9928,236.9760,442.8463,1.1015,23.0525,2.2061,0.8040
Stage 5,42.6205,22.0609,-20.1610,59.4255,0.2230,76.1951,2655.9757,291.4741,226.4117,208.3459,91.7563,241.1899,445.1155,1.0984,28.8914,2.0592,0.7768
